!["Movies"](Images\movie.jpg)

# Movie Recommendation System

## Business Problem/Question

## EDA

In [5]:
# Importing packages
import numpy as np
import pandas as pd 
import math
import surprise
from surprise.prediction_algorithms import *
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.dummy import DummyClassifier

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree

from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import roc_curve, roc_auc_score, plot_roc_curve
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

### Read in data

### Missingness

Next we look at null values to try to determine which columns might need to be imputed or if the data is too incomplete to be useful. 

In [ ]:
# Looking at the total nulls left in remaining columns

df.info()

## Feature Engineering

### Feature Engineering

## Initial Modeling

### Initial Dummy Model

#### Decide Xs/y

#### Train/Test Split

#### Modeling 

### Initial Model

#### Decide Xs/y

#### Train/Test Split

#### Preprocessing Steps (SS, OHE, SI)

#### Modeling 

## Evaluation of Initial Models

### Scores of Initial Models

## New Models

### New Model

#### Decide Xs/y

#### Train/Test Split

#### Preprocessing Steps (SS, OHE)

#### Modeling 

## Final Model Evaluations 

### Score of New Models

### Assessment